<a href="https://colab.research.google.com/github/fercouri/Calculo-Numerico/blob/main/Trabalho_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importações

In [ ]:
from IPython.display import clear_output
import numpy as np
import sympy as sp

# Resolução de sistemas n ao-lineares para o cálculo de pontos de integração da quadratura de Gauss-Legendre

---

Objetivo:
Desenvolver um código computacional para calcular pontos de integração da quadratura de Gauss em qualquer intervalo.

## 1. Definir o domínio de integração [a, b]

---




In [ ]:
def lerIntervaloIntegração():
    while True:
        try:
            # print('Dado o intervalo [a, b], qual o valor de a?')
            # a = int(input())
            # print('E o valor de b?')
            # b = int(input())
            a, b = 1, 3
            return a, b
        except ValueError:
            print("Tipo inválido, por favor insira números inteiros válidos.")

## 2. Definir número de pontos de integração N

---



In [ ]:
def lerQuantidadePontosIntegracao():
  while True:
    try:
      # print('Qual o número de pontos N?')
      # N = int(input())
      N = 3

      return N
    except ValueError:
            print("Tipo inválido, por favor insira números inteiros válidos.")

## 3. Inicializar pesos w e pontos t com aproximações iniciais.

---



Detalhes de implementação: Nota-se que como o índice começa no 0, tivemos que adicionar i + 1 nas operações, para que pudessem ser realizadas sem problemas

###Cálculo dos Pesos $w^0_i$

* Para $𝑖 ≤ \frac{N}{2}$​, temos:
  $$w^0_i = \frac{b-a}{2N}i$$

* Para $𝑖 > \frac{N}{2}$​, temos:
  $$w^0_i = w^0_{\frac{N}{2}-i} $$ isso representa espelhamento, ou seja, para índices maiores que $\frac{N}{2}$, os valores de $w_0$ se igualam aos valores da primeira metade do vetor ($w^0_i = w^0_{\frac{N}{2} - (i - \frac{N}{2})}$)

---



In [ ]:
def calculoPesosW0(a, b, N):
  w_0 = np.zeros(N)

  for i in range(0, N // 2):
    # print(i, ": ",(b - a), ", ", (2 * N), " * ", (i + 1), " = ", ((b - a) / (2 * N)) * (i + 1))
    w_0[i] = ((b - a) / (2 * N)) * (i + 1)

  for i in range(N // 2, N):
    index = (N // 2) - ((i + 1) - (N // 2))

    if index < 0:
      index = index * (-1)

    # print(i, ": ", N // 2, "- (",  i + 1, " - ", N // 2, ") = |", index, "| --> ", w_0[index])

    w_0[i] = w_0[index]

  return w_0

print("---------------------------------------------------")
print("TESTE -> Saída esperada: [0.25 0.5  0.5  0.25]")
w_0 = calculoPesosW0(-1, 1, 4)
print("Pesos w_0:", w_0)
print("---------------------------------------------------")

---------------------------------------------------
TESTE -> Saída esperada: [0.25 0.5  0.5  0.25]
Pesos w_0: [0.25 0.5  0.5  0.25]
---------------------------------------------------


### Cálculo dos Pesos $t^0_i$

* Para $𝑖 ≤ \frac{N}{2}$​, temos:
  $$t^0_i = a + \frac{i w_0[i]}{2}$$

* Para $𝑖 > \frac{N}{2}$​, temos:
   $$t^0_i = (a + b) - t^0_{\frac{N}{2}-i}$$
   
   ou seja, para índices maiores que $\frac{N}{2}$, os valores de $t_0$ são $t^0_i = (a + b) - t_0 [\frac{N}{2} - (i- \frac{N}{2})]$

* Para N ímpar, $t^0_\frac{N}{2} = \frac{(a + b)}{2} $

---



In [ ]:
def calculoPesosT0(a, b, N, w0):
  t_0 = np.zeros(N)

  for i in range(0, N // 2):
    t_0[i] = a + (((i + 1) * w_0[i]) / 2)

  for i in range(N // 2, N):
    index = (N // 2) - ((i + 1) - (N // 2))

    if index < 0:
      index = index * (-1)

    t_0[i] = (a + b) - t_0[index]

  # Caso N ímpar: Definindo o ponto médio
  if N % 2 != 0:
    t_0[N // 2] = (a + b) / 2

  return t_0

print("---------------------------------------------------")
print("TESTE -> Saída esperada: [−0.875, −0.5, 0.5, 0.875]")
w_0 = calculoPesosW0(-1, 1, 3)
t_0 = calculoPesosT0(-1, 1, 3, w_0)
print("Pesos t_0:", t_0)
print("---------------------------------------------------")

---------------------------------------------------
TESTE -> Saída esperada: [−0.875, −0.5, 0.5, 0.875]
Pesos t_0: [-0.83333333  0.         -0.83333333]
---------------------------------------------------


## 4. Calcular o sistema de equações F(w, t)

---
Trata-se de encontrar $w_i$ e $t_i$, tal que:
$$
\sum_{i=1}^{N} w_i \cdot (t_i)^{j-1} = \int_{a}^{b} x^{j-1} \, dx \quad \text{para} \quad j = 1, 2, \dots, 2N
$$

Como estipulado no enunciado, as integrais do lado direito da equação devem ser tratadas de forma numérica usando uma regra de Newton-Cotes repetida com m=1000 partições.

### Newton-Cotes
Utilizaremos Regra 1/3 de Simpson,
$$
\int_a^b f(x) \, dx \approx \frac{h}{3} (\sum_{i=0}^{m} c_i f(x_i))
$$

Onde:
- $c_0 = c_m = 1$
- caso contrário (i = 1, ..., m - 1):
 * $c_i = 4$, se i for ímpar

 * $c_i = 2$, se i for par

- $h = \frac{b - a}{m}$ é a largura de cada subintervalo.



In [ ]:
def integral_simpson(a, b, m, j):
    # Função a ser integrada: f(x) = x^(j-1)
    def f(x):
        return x**(j-1)

    h = (b - a) / m

    # Como c0 = cm = 1
    integral = f(a) + f(b)

    for i in range(1, m, 2):
        integral += 4 * f(a + i * h)

    for i in range(2, m, 2):
        integral += 2 * f(a + i * h)

    integral *= h / 3

    return integral

### Cálculo do Sistema de Equações

---
Assim, para a aplicação do método de Newton reescrevemos como:
$$
\sum_{i=1}^{N} w_i(t_i)^{j-1} - g_j = 0, \quad j = 1, 2, \dots, 2N
$$

onde $ g_j $ é dado por:

$$
g_j = \int_a^b x^{j-1} \, dx
$$



In [ ]:
def calculoF(w_0, t_0, a, b, N):
    F_val = np.zeros(2 * N)

    for j in range(1, 2 * N + 1):
        # Cálculo da integral aproximada usando a regra de Simpson 1/3 com m subintervalos
        m = 1000
        integral = integral_simpson(a, b, m, j)

        # Somatório para calcular a equação para cada j
        # somatorio = np.sum(w_0 * (t_0**(j-1)))
        somatorio = sum(w_0[i] * (t_0[i] ** (j - 1)) for i in range(N))

        # Subtrai a integral da soma ponderada
        F_val[j - 1] = somatorio - integral

    return F_val

## 5. Montar a matriz Jacobiana J(w,t)

---

Montar a matriz jacobiana J(w,t) formada pela derivada da função $f_j(w_i, t_i)$ pelas variáveis $w_i$ e $t_i$, dada por:

$$
J(w, t) =
\begin{bmatrix}
\frac{\partial f_1}{\partial w_1} & \cdots & \frac{\partial f_1}{\partial w_N} & \frac{\partial f_1}{\partial t_1} & \cdots & \frac{\partial f_1}{\partial t_N} \\
\frac{\partial f_2}{\partial w_1} & \cdots & \frac{\partial f_2}{\partial w_N} & \frac{\partial f_2}{\partial t_1} & \cdots & \frac{\partial f_2}{\partial t_N} \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
\frac{\partial f_{2N}}{\partial w_1} & \cdots & \frac{\partial f_{2N}}{\partial w_N} & \frac{\partial f_{2N}}{\partial t_1} & \cdots & \frac{\partial f_{2N}}{\partial t_N}
\end{bmatrix}
$$


Assim, definindo uma perturbação $ \epsilon = 10^{-8} $, podemos aproximar as derivadas parciais:

$$
\frac{\partial f_i}{\partial w_j} \approx \frac{f_i(w_1 + \epsilon, w_2, \dots, w_N, t_1, \dots, t_N) - f_i(w_1, w_2, \dots, w_N, t_1, \dots, t_N)}{\epsilon}
$$

e, semelhantemente, ocorre para a derivada $f_i$ em relação a $t_j$.

In [ ]:
def calculoJ(w, t, a, b, N, F0, m, eps):
    J = np.zeros((2 * N, 2 * N))

    # Aproximação das derivadas parciais em relação a w
    for j in range(N):
        w_pert = np.copy(w)
        w_pert[j] += eps  # perturba somente w[j]
        F_pert = calculoF(w_pert, t, a, b, N)
        # Diferença finita para todas as funções f_i (i = 0,1,...,2N-1)
        J[:, j] = (F_pert - F0) / eps

    # Aproximação das derivadas parciais em relação a t
    for j in range(N):
        t_pert = np.copy(t)
        t_pert[j] += eps  # perturba somente t[j]
        F_pert = calculoF(w, t_pert, a, b, N)
        # Armazena a derivada no bloco correspondente (colunas de N a 2N-1)
        J[:, N + j] = (F_pert - F0) / eps

    return J

## 6. Resolver pelo método de Newton

---
O método de Newton é iterativo e, a cada iteração, atualiza as aproximações dos
w e t de acordo com a solução do sistema linear:

$$
J(w,t)⋅s=−F(w,t)
$$

onde
- s é o vetor de correção que é dividido em duas partes (primeiros N componentes para w e os restantes para t).

Aqui optamos por resolver por mínimos quadrados (np.linalg.lstsq), quando a matriz J for singular, apresentando um resultado aproximado.

In [ ]:
def newton(w0, t0, a, b, N, tol=1e-8, max_iter=100):
    w = np.copy(w0)
    t = np.copy(t0)

    for k in range(max_iter):
        F_val = calculoF(w, t, a, b, N)
        J_val = calculoJ(w, t, a, b, N, F_val, m=1000, eps=1e-8)

        try:
            s = np.linalg.solve(J_val, -F_val)
        except ValueError as e:
            # print("Matriz Jacobiana singular -> não existe solução para o sistema linear Js=−F")
            s = np.linalg.lstsq(J_val, -F_val, rcond=None)[0]

        # Critério de parada: norma infinito do vetor s
        if np.linalg.norm(s, ord=np.inf) < tol:
            w = w + s[:N]
            t = t + s[N:]
            # print(f"Convergência atingida após {k+1} iterações.")
            return w, t

        # Atualiza as aproximações
        w = w + s[:N]
        t = t + s[N:]

    # print("Número máximo de iterações alcançado.")
    return w, t

# newton([0.25, 0.5, 0.5, 0.25], [-0.875, -0.5, 0.5, 0.875], -1, 1, 4, 1e-8, 100)

# 7. Execução

---



In [ ]:
a, b = lerIntervaloIntegração()
N = lerQuantidadePontosIntegracao()

In [ ]:
w_0 = calculoPesosW0(a, b, N)
t_0 = calculoPesosT0(a, b, N, w_0)

In [ ]:
w_sol, t_sol = newton(w_0, t_0, a, b, N, tol=1e-8, max_iter=100)


clear_output(wait=True)

print("O intervalo é: [", a ,",", b, "]")
print("O número de pontos é: ", N)
print("w_0 inicial é:", w_0)
print("t_0 inicial é:", t_0)
print("----------------------------------------")
print("Solução aproximada dos pontos:", t_sol)
print()
print("Solução aproximada dos pesos:", w_sol)

O intervalo é: [ 1 , 3 ]
O número de pontos é:  3
w_0 inicial é: [0.33333333 0.33333333 0.33333333]
t_0 inicial é: [1.16666667 2.         1.16666667]
----------------------------------------
Solução aproximada dos pontos: [-13.72660727   2.01829437 -13.72660727]

Solução aproximada dos pesos: [-1.44833217e+12  1.99646852e+00  1.44833217e+12]


### Funções auxiliares

In [ ]:
def integral_exata(f, a, b):
    x = sp.symbols('x')
    f_expr = f(x)

    integral = sp.integrate(f_expr, (x, a, b))

    return float(integral.evalf())

In [ ]:
def integral_gauss_legendre(f, a, b, w, t):
    # Mapeia os pontos de Gauss-Legendre de [-1, 1] para [a, b]
    midpoint = 0.5 * (b + a)
    half_length = 0.5 * (b - a)

    integral_sum = 0.0
    for i in range(len(t)):
        # Calcula a função no ponto mapeado
        x = midpoint + half_length * t[i]
        integral_sum += w[i] * f(x)

    # Multiplica pela largura do intervalo
    return half_length * integral_sum

### Definindo intervalos, números de pontos e funções a serem testadas

In [ ]:
a, b = -1, 1

N = [2, 3, 4, 5, 6]

# Funções para testar
def f1(x):
    return x**2

def f2(x):
    return 2*x

def f3(x):
    return sp.sin(x)

### Criando Tabela 1:

Usa a, b e N

In [ ]:
def gerar_tabela_pesos_pontos(a, b, Ns):
    print(f"\nIntervalo: [{a}, {b}]")
    print("| N |       Pesos (w)       |       Pontos (t)       |")
    print("| - | --------------------- | ---------------------- |")

    for N in Ns:
        # Calcular pesos e pontos iniciais
        w_0 = calculoPesosW0(a, b, N)
        t_0 = calculoPesosT0(a, b, N, w_0)

        # Refinar pesos e pontos usando o método de Newton
        w_sol, t_sol = newton(w_0, t_0, a, b, N, tol=1e-8, max_iter=100)

        print(f"| {N} | {w_sol} | {t_sol} |")

In [ ]:
print("\nTabela 1: Valores dos pesos e pontos de integração para o intervalo [a, b]")
gerar_tabela_pesos_pontos(a, b, N)


Tabela 1: Valores dos pesos e pontos de integração para o intervalo [a, b]

Intervalo: [-1, 1]
| N |       Pesos (w)       |       Pontos (t)       |
| - | --------------------- | ---------------------- |
| 2 | [1. 1.] | [-0.57735027  0.57735027] |
| 3 | [-4.03633422e+05  3.31911624e-02 -5.66638713e-12] | [ 1.36591370e+02  2.51294111e+00 -3.20596986e+05] |
| 4 | [0.34785485 0.65214515 0.65214515 0.34785485] | [-0.86113631  0.33998104 -0.33998104  0.86113631] |
| 5 | [0.23692688 0.47862867 0.56888889 0.23692688 0.47862867] | [-9.06179846e-01 -5.38469310e-01  2.49876856e-15  9.06179846e-01
  5.38469310e-01] |
| 6 | [ 1.21600034e-14  1.91351349e-07  1.59823438e-02  3.28783918e-01
 -4.81704209e+01  0.00000000e+00] | [-9.07374963e+01 -1.73349898e+01  2.90653550e+00 -2.63954166e+00
  9.00071756e-01  2.77395335e+05] |


### Criando Tabela 2:
Usa funções e N

In [ ]:
# Intervalos e soluções exatas
testes = [
    {"funcao": f1, "a": -1, "b": 1, "exata": integral_exata(f1, -1, 1)},
    {"funcao": f2, "a": 0, "b": 1, "exata": integral_exata(f2, 0, 1)},
    {"funcao": f3, "a": 0, "b": np.pi, "exata": integral_exata(f3, 0, np.pi)}
]

print("\nTabela 2: Comparação exata e aproximada da integral da função f(x) no intervalo [a, b]")
for teste in testes:
    print(f"\nFunção: {teste['funcao'].__name__}, Intervalo: [{teste['a']}, {teste['b']}]")
    print("| N |     Solução Exata    |       Solução Aproximada       |    Erro    |")
    print("| - | -------------------- | ------------------------------ | ---------- |")
    for Ns in N:
        # Calcular pesos e pontos iniciais
        w_0 = calculoPesosW0(teste['a'], teste['b'], Ns)
        t_0 = calculoPesosT0(teste['a'], teste['b'], Ns, w_0)

        # Refinar pesos e pontos usando o método de Newton
        w_sol, t_sol = newton(w_0, t_0, teste['a'], teste['b'], Ns, tol=1e-8, max_iter=100)

        # Calcular a integral aproximada
        integral_aprox = integral_gauss_legendre(teste['funcao'], teste['a'], teste['b'], w_sol, t_sol)

        # Calcular o erro
        erro = abs(teste['exata'] - integral_aprox)

        # Exibir resultados
        print(f"| {Ns} | {teste['exata']} | {integral_aprox} | {erro} |")


Tabela 2: Comparação exata e aproximada da integral da função f(x) no intervalo [a, b]

Função: f1, Intervalo: [-1, 1]
| N |     Solução Exata    |       Solução Aproximada       |    Erro    |
| - | -------------------- | ------------------------------ | ---------- |
| 2 | 0.6666666666666666 | 0.6666666666666663 | 3.3306690738754696e-16 |
| 3 | 0.6666666666666666 | -7530670468.747863 | 7530670469.41453 |
| 4 | 0.6666666666666666 | 0.6666666666666663 | 3.3306690738754696e-16 |
| 5 | 0.6666666666666666 | 0.6666666666666663 | 3.3306690738754696e-16 |
| 6 | 0.6666666666666666 | 0.6666666666666663 | 3.3306690738754696e-16 |

Função: f2, Intervalo: [0, 1]
| N |     Solução Exata    |       Solução Aproximada       |    Erro    |
| - | -------------------- | ------------------------------ | ---------- |
| 2 | 1.0 | 0.7500000000000001 | 0.2499999999999999 |
| 3 | 1.0 | 1.1774334133250643e+35 | 1.1774334133250643e+35 |
| 4 | 1.0 | 0.75 | 0.25 |
| 5 | 1.0 | 0.7500000000000001 | 0.2499999999999